In [1]:
import pandas as pd
import numpy as np

import librosa, librosa.display
import matplotlib.pyplot as plt

import matplotlib.axes as axes
from pydub import AudioSegment
from pydub.playback import play
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import  Dense,Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


c:\Users\2maju\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
data = pd.read_csv('data.csv')

In [3]:
sample=data['File'].tolist()
output=data['Output'].tolist()
print(len(data))
print(len(sample))
sample_cp=sample



488
488


In [4]:

for file_path in sample:
    t1 = 0  # Start time in milliseconds
    t2 = 10 * 1000  # End time in milliseconds (10 seconds)

    # Load the original audio
    old_audio = AudioSegment.from_wav(file_path)

    # Extract the desired portion of the audio
    new_audio = old_audio[t1:t2]

    # Create a new file name for the cut audio
    new_file_path = file_path.replace(".wav", "_cut.wav")

    # Save the cut audio to a new file
    new_audio.export(new_file_path, format="wav")

    # Delete the original audio file
    os.remove(file_path)

    # Rename the cut audio file to the original file name
    os.rename(new_file_path, file_path)


In [5]:
def compute_spectrogram(file_path):
    signal, sample_rate = librosa.load(file_path, sr=22050)
    h_l = 512
    nfft = 2048
    stft = librosa.stft(y=signal, n_fft=nfft, hop_length=h_l)
    log_stft = librosa.amplitude_to_db(np.abs(stft))
    return log_stft

csv_file_path = 'data.csv'  # Path to your CSV file
column_name = 'File'  # Name of the column containing file paths

df = pd.read_csv(csv_file_path)
file_paths = df[column_name].tolist()

array_of_spectrograms = []
for file_path in file_paths:
    spectrogram = compute_spectrogram(file_path)
    array_of_spectrograms.append(spectrogram)

# Print the shapes of the spectrograms
# for spectrogram in array_of_spectrograms:
#     print(spectrogram)
print(array_of_spectrograms[0].shape)
print(array_of_spectrograms[0])

(1025, 431)
[[ 10.792866   12.868214   11.458997  ...  18.315292   16.638779
   14.243755 ]
 [  9.226515   10.022765   -4.494703  ...  13.691319    9.945406
   10.235294 ]
 [  4.462182    2.1856666   4.405513  ...   2.7809117   4.7742753
    5.1901193]
 ...
 [-49.74349   -49.74349   -49.74349   ... -49.74349   -49.74349
  -49.74349  ]
 [-49.74349   -49.74349   -49.74349   ... -49.74349   -49.74349
  -49.74349  ]
 [-49.74349   -49.74349   -49.74349   ... -49.74349   -49.74349
  -49.74349  ]]


In [6]:
np_array_of_stft = np.array(array_of_spectrograms)

np_array_of_stft.shape

(488, 1025, 431)

In [7]:
array_of_log_spectrograms = []
for spectrogram in array_of_spectrograms:
    # Normalize the spectrogram
    normalized_spectrogram = (spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram))

    # Apply the log transformation
    log_spectrogram = np.log(normalized_spectrogram + 1e-10)  # Adding a small value to avoid log(0) error

    array_of_log_spectrograms.append(log_spectrogram)


In [8]:
np_array_of_stft_log = np.array(array_of_log_spectrograms)
np_array_of_stft_log.shape

(488, 1025, 431)

In [9]:
labels=np.array(output)
# reshaped_spectrograms = np_array_of_stft_log.reshape((np_array_of_stft_log.shape[0], -1))
print(labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [10]:
# from sklearn.model_selection import train_test_split


# X_train, X_test, y_train, y_test = train_test_split(np_array_of_stft_log, labels, test_size=0.4, random_state=0)


# # Create the model
# model = Sequential([
#     Dense(16, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2] )),
#     MaxPooling2D((2, 2)),
   
#     Dense(16, activation='relu'),
#     Dropout(0.2),
#     # Flatten(),
#     Dense(8, activation='relu'),
#     Dropout(0.2),
#     Dense(8, activation='relu'),
#     Dense(8, activation='relu'),
    
#       Dropout(0.2),
#     Dense(4, activation='relu'),
#     Dense(4, activation='relu'),
#       Dropout(0.2),
#     Dense(4, activation='relu'),
#     Dense(4, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# # Train the model
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# # Evaluate the model
# loss, accuracy = model.evaluate(X_test, y_test)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np_array_of_stft_log, labels, test_size=0.3, random_state=42)

# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(np_array_of_stft_log.shape[1], np_array_of_stft_log.shape[2], 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

Epoch 1/10
11/11 [==============================] - 88s 7s/step - loss: 304.7257 - accuracy: 0.5191 - val_loss: 8.0074 - val_accuracy: 0.4150
Epoch 2/10
11/11 [==============================] - 66s 6s/step - loss: 2.7380 - accuracy: 0.7625 - val_loss: 0.4307 - val_accuracy: 0.8571
Epoch 3/10
11/11 [==============================] - 63s 6s/step - loss: 0.3888 - accuracy: 0.8856 - val_loss: 0.3240 - val_accuracy: 0.8435
Epoch 4/10
11/11 [==============================] - 62s 6s/step - loss: 0.2384 - accuracy: 0.8886 - val_loss: 0.3085 - val_accuracy: 0.8707
Epoch 5/10
11/11 [==============================] - 62s 6s/step - loss: 0.1485 - accuracy: 0.9267 - val_loss: 0.2863 - val_accuracy: 0.8844
Epoch 6/10
11/11 [==============================] - 65s 6s/step - loss: 0.0895 - accuracy: 0.9824 - val_loss: 0.2860 - val_accuracy: 0.8776
Epoch 7/10
11/11 [==============================] - 64s 6s/step - loss: 0.0393 - accuracy: 0.9941 - val_loss: 0.3335 - val_accuracy: 0.8980
Epoch 8/10
11/11 [